In [ ]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Dropdown, VBox, HBox, HTML, Output

alt.data_transformers.enable("vegafusion")
full_v_dem_data = pd.read_csv("../../data/raw/V-Dem-CY-Full+Others-v15.csv", low_memory = False)
full_v_dem_data['year'] = pd.to_datetime(full_v_dem_data['year'], format = '%Y')

columns_to_keep = ['country_name',
                   'year',
                   'e_regionpol_7C',
                   'e_regiongeo',

                   'v2x_polyarchy',
                   'v2x_libdem',
                   'v2x_partipdem',
                   'v2x_delibdem',
                   'v2x_egaldem',
                   ]

data = full_v_dem_data[columns_to_keep]

dict_for_renaming = {

    # General 
    "country_name": "Country",
    "year": "Year",
    "e_regionpol_7C": "Politico-Geographical Region",
    "e_regiongeo": "Region",

    # High-level indices
    "v2x_polyarchy": "Electoral Democracy Index",
    "v2x_libdem": "Liberal Democracy Index",
    "v2x_partipdem": "Participatory Democracy Index",
    "v2x_delibdem": "Deliberative Democracy Index",
    "v2x_egaldem": "Egalitarian Democracy Index",
    
}

high_level_indices = [
    "Electoral Democracy Index",
    "Liberal Democracy Index",
    "Participatory Democracy Index",
    "Deliberative Democracy Index",
    "Egalitarian Democracy Index"
]

data = data.rename(columns = dict_for_renaming)

data["Year_dt"] = pd.to_datetime(data["Year"]) # dtype: datetime
data["Year_int"] = pd.to_datetime(data["Year"]).dt.year.astype(int) # dtype: int64


In [13]:
import ipywidgets as widgets
from IPython.display import display

high_level_indices = [
    "Electoral Democracy Index",
    "Liberal Democracy Index",
    "Participatory Democracy Index",
    "Deliberative Democracy Index",
    "Egalitarian Democracy Index"
]

subset = data[data["Year_int"] >= 1900].copy()
subset["Standard Deviation Across Democracy Indices"] = (
    subset[high_level_indices].std(axis=1)
)

min_year = int(subset["Year_int"].min())
max_year = int(subset["Year_int"].max())

base = (
    alt.Chart(subset)
    .transform_fold(high_level_indices, as_=["dimension", "score"])
    .transform_calculate(
        country_year="datum.Country + ', ' + toString(datum.Year_int)"
    )
    .encode(
        x=alt.X(
            "dimension:N",
            title="Democracy Index",
            axis=alt.Axis(
                domain=False,
                titleFontSize=16,
                titlePadding=20,
                labelAlign="center",
                labelAngle=0,
                labelPadding=30,
                labelFontSize=14,
                labelExpr="{ 'Electoral Democracy Index': 'Electoral', "
                          "'Liberal Democracy Index': 'Liberal', "
                          "'Participatory Democracy Index': 'Participatory', "
                          "'Deliberative Democracy Index': 'Deliberative', "
                          "'Egalitarian Democracy Index': 'Egalitarian' }[datum.value]"
            )
        ),
        y=alt.Y(
            "score:Q",
            title="Index Value",
            scale=alt.Scale(domain=[0, 1]),
            axis=alt.Axis(
                titleFontSize=16,
                titlePadding=30,
                labelFontSize=14,
                labelPadding=10,
                grid=False,
                domainColor="#dddddd"
            )
        ),
        color=alt.Color(
            "Country:N",
            title="Country",
            legend=alt.Legend(
                title="Country",
                labelFontSize=14,
                titlePadding=30,
                titleFontSize=16
            ),
            scale=alt.Scale(range=["#475569", "#94A3B8"])
        ),
        detail=["Country:N", "Year_int:O"],
        tooltip=[
            alt.Tooltip("Year_int:O", title="Year"),
            alt.Tooltip("score:Q", title="Index Value", format=".2f"),
        ]
    )
    .properties(width=800)
)

dims_df = pd.DataFrame({"dimension": high_level_indices})

vlines = (
    alt.Chart(dims_df)
    .mark_rule(stroke="#AAAAAA", strokeWidth=3, opacity=0.5)
    .encode(x=alt.X("dimension:N"))
)

sd_data = data[data["Year_int"] >= 1900].copy()
sd_data["Standard Deviation Across Democracy Indices"] = sd_data[high_level_indices].std(axis=1)
sd_data["country_year"] = sd_data["Country"] + ", " + sd_data["Year_int"].astype(str)

instructions_html = widgets.HTML(
    value="""
    <div style='text-align:left; margin-top:10px; margin-bottom:50px;'>
        <span style="font-size:16px;">
            <b>Select countries</b> using the dropdown menus. <b>Drag the slider</b> to select a year.<br>
            You can also click a year in the heatmap or line chart to adjust the selection.</br>
            <b>Shift-click</b> for multiple years. <b>Click outside</b> lines in bottom plot to show all years.</br>
            
        </span>
    </div>
    """
)

def make_dashboard(country1: str, country2: str, initial_year: int):
   
    year_sel = alt.selection_point(
        fields=["Year_int"],
        on="click",
        empty="all",
        value=[{"Year_int": initial_year}] 
    )

    year_label_sel = alt.selection_point(
        fields=["Year_int"],
        name="year_label_sel",
        empty="none",
        value=[{"Year_int": initial_year}] 
    )

    line1 = (
        base
        .transform_filter(alt.datum.Country == country1)
        .mark_line(strokeWidth=2)
        .transform_filter(year_sel)
    )

    dots1 = (
        base
        .transform_filter(alt.datum.Country == country1)
        .mark_circle(size=75, opacity=1)
        .transform_filter(year_sel)
    )

    # Country 2
    line2 = (
        base
        .transform_filter(alt.datum.Country == country2)
        .mark_line(strokeWidth=2)
        .transform_filter(year_sel)
    )

    dots2 = (
        base
        .transform_filter(alt.datum.Country == country2)
        .mark_circle(size=75, opacity=1)
        .transform_filter(year_sel)
    )

    layer1 = line1 + dots1
    layer2 = line2 + dots2

    top_chart = (
        (vlines + layer1 + layer2)
        .properties(
            width=700,
            height=300,
            title=alt.TitleParams(
                text="Democracy Index Performance",
                fontSize=20,
                offset=30
            )
        )
        .add_params(year_sel, year_label_sel)
    )

    base_heat = (
        alt.Chart(sd_data)
        .mark_rect()
        .transform_filter(
            alt.FieldOneOfPredicate(field="Country", oneOf=[country1, country2])
        )
        .encode(
            x=alt.X(
                "Year_int:O",
                title="Year",
                axis=alt.Axis(
                    labelAngle=50,
                    titleFontSize=16,
                    titlePadding=20,
                    labelPadding=10,
                    labelFontSize=14,
                    labelExpr="datum.value % 10 == 0 ? datum.label : ''"
                )
            ),
            y=alt.Y(
                "Country:N",
                title="Country",
                sort="ascending",
                axis=alt.Axis(
                    titleFontSize=16,
                    labelFontSize=14
                )
            ),
            color=alt.Color(
                "Standard Deviation Across Democracy Indices:Q",
                title="Standard Deviation",
                scale=alt.Scale(scheme="magma"),
                legend=alt.Legend(
                    title="Standard Deviation",
                    labelFontSize=14,
                    titleFontSize=16,
                    titlePadding=10,
                    gradientLength=200,
                    format=".02f",
                    offset=90
                )
            ),
            opacity=alt.condition(year_sel, alt.value(1.0), alt.value(0.7)),
            tooltip=[
                alt.Tooltip("Year_int:O", title = "Year"),
                alt.Tooltip(
                    "Standard Deviation Across Democracy Indices:Q",
                    title="Standard deviation",
                    format=".2f"
                )
            ]
        )
    )

    labels = (
        alt.Chart(sd_data)
        .mark_text(fontSize=14, baseline="middle", align="center")
        .transform_filter(
            alt.FieldOneOfPredicate(field="Country", oneOf=[country1, country2])
        )
        .transform_filter(year_sel)   
        .encode(
            x=alt.X("Year_int:O"),
            y=alt.Y("Country:N"),
            text=alt.Text("country_year:N"),
            color=alt.value("black"),
            opacity=alt.condition(year_label_sel, alt.value(1), alt.value(0))
        )
    )

    stdev_heat = (
        (base_heat + labels)
        .properties(
            width=700,
            height=280,
            title=alt.TitleParams(
                text="Standard Deviation Across Democracy Indices",
                fontSize=20,
                offset=40
            ),
        )
        .add_params(year_sel, year_label_sel)
    )

    final_chart = (
        alt.vconcat(
            stdev_heat,
            top_chart,
            spacing=50
        )
        .properties(
            padding={"top": 50}
        )
    )

    final = final_chart.configure_view(stroke=None)
    return final

title_html = widgets.HTML(
    value="""
    <div style='width:100%; text-align:left;'>
        <h1 style="font-size:40px; margin-bottom:40px; margin-left:240px; margin-top: 200px">
            Dimensions of Democracy
        </h1>
    </div>
    """
)

country_options = sorted(subset["Country"].dropna().unique().tolist())

country1_dropdown = widgets.Dropdown(
    options=country_options,
    value="China",
    description="Country 1:",
    layout=widgets.Layout(width="250px")
)

country2_dropdown = widgets.Dropdown(
    options=country_options,
    value="China",
    description="Country 2:",
    layout=widgets.Layout(width="250px")
)

year_slider = widgets.IntSlider(
    value=max_year,
    min=min_year,
    max=max_year,
    step=1,
    description="Year:",
    continuous_update=False,
    layout=widgets.Layout(width="520px")
)

out = widgets.Output()

def update_dashboard(change=None):
    with out:
        out.clear_output()
        chart = make_dashboard(
            country1_dropdown.value,
            country2_dropdown.value,
            year_slider.value    
        )
        display(chart)


country1_dropdown.observe(update_dashboard, names="value")
country2_dropdown.observe(update_dashboard, names="value")
year_slider.observe(update_dashboard, names="value")


update_dashboard()

dropdowns_box = widgets.HBox(
    [country1_dropdown, country2_dropdown],
    layout=widgets.Layout(
        justify_content="flex-start",
        width="50%",
        margin="0px 0px 0px 0px"
    )
)

controls_box = widgets.VBox(
    [dropdowns_box, year_slider],
    layout=widgets.Layout(
        align_items="flex-start",
        width="100%",
        margin="0px 0px 0px 0px"
    )
)

main_box = widgets.VBox(
    [title_html, instructions_html, controls_box, out],
    layout=widgets.Layout(margin="0", padding="0 0 0 200px", align_items="flex-start")
)
display(main_box)
